In [1]:
import polars as pl
from neo4j import GraphDatabase
from eyecite import get_citations
from eyecite.models import FullCaseCitation, ShortCaseCitation
import polars as pl


URI = "bolt://localhost:7687"
AUTH = ("neo4j", "fairusecases")

In [2]:
## Get all case law
wl = pl.concat([pl.read_csv("./Cases/WestLawMatch.csv"), pl.read_csv("./Cases/WestLawNotMatch.csv")])
## All fair use cases
fair_use = [c for c in wl["Citation"]]

## Get Fair Use Case Matches
def get_fair_use_citations(doc, fair_use = fair_use):

    citations = [c.matched_text() for c in get_citations(doc) if type(c) is FullCaseCitation or type(c) is ShortCaseCitation]

    cited = set(citations).intersection(set(fair_use))

    return cited

In [3]:
df = pl.read_csv("./Cases/Incomplete_Opinion.csv")

In [4]:
## Get citations
df = df.with_columns(
    pl.col("Opinion").map_elements(get_fair_use_citations).alias("CitedCases")
)

## Obtain "adjacency list" of cases and their cited counterparts
df_citation = df.select(["Title", "CitedCases"])

df_citation_rel = df_citation.with_columns(
                pl.col("CitedCases").map_elements(list)
                ).explode("CitedCases").unique().sort("Title")

## Create a map between the citation and the Title of the case based on Westlaw's name
name_citation_map = wl.select(["Title", "Citation"]).unique().rename({"Title": "CitedCaseName"})

In [5]:
df_citation_rel = df_citation_rel.rename({"CitedCases": "Citation"}).join(name_citation_map, on = ["Citation"], how = "inner")

df_citation_rel.write_csv("./Cases/IncompleteOpinion_CitationRelationship.csv")

In [8]:
obs = df.select(["Title", "Opinion"]).iter_rows(named=True)

def update_incomplete_cases(tx, obs):
    result = tx.run("""
        MATCH (c: Case {WestLawCaseName: $westlaw})-[:HAS_OPINION]-(o:Opinion)
        SET o.Document = $document
        """, westlaw = obs["Title"], document = obs["Opinion"]
    )

    return result

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
         for ob in obs:
            session.execute_write(update_incomplete_cases, ob)

In [9]:
def update_case_citation(tx, obs):
    result = tx.run("""
        MATCH (c1: Case {WestLawCaseName: $citer}), (c2: Case {WestLawCaseName: $citee})
        WITH c1, c2
        MERGE (c1)-[:CITED {Citation: $Citation}]->(c2)
        """, citer = obs["Title"], citee = obs["CitedCaseName"], Citation = obs["Citation"]
    )
    return result

obs = df_citation_rel.iter_rows(named=True)
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
         for ob in obs:
            if ob["Title"] != ob["CitedCaseName"]:
                session.execute_write(update_case_citation, ob)